In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry
import geopandas as gpd
import pandas as pd
from shapely import wkt
import psycopg2

In [2]:
# import packages
import requests
import json
import pandas as pd
import geopandas as gpd

In [3]:
pip install GeoAlchemy2 SQLAlchemy psycopg2

Note: you may need to restart the kernel to use updated packages.


# Creating dataframe of API

In [4]:
# send the request
def jason2dataf(url):
    response = requests.get(url)
    response
    # store the raw text of the response in a variable
    raw_data = response.text
    raw_data
    # parse the raw text response into a JSON
    data = json.loads(raw_data)
    data
    # from JSON to Pandas DataFrame
    data_df = pd.json_normalize(data)
    return data_df

In [5]:
#dataf_liguria = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/7')
dataf_lombardia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/3')
#dataf_Piemonte = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/1')
#dataf_valle = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/regioni/2')
data_prov_milan = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/15')
data_prov_varese = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/12')
data_prov_como = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/13')
data_prov_lecco = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/97')
data_prov_sondrio = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/14')
data_prov_bergamo = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/16')
data_prov_brescia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/17')
data_prov_pavia = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/18')
data_prov_cremona = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/19')
data_prov_mantova = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/20')
data_prov_lodi = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/98')
data_prov_Monza = jason2dataf('https://test.idrogeo.isprambiente.it/api/pir/province/108')



In [6]:
# Merge the DataFrames
merged_df = pd.concat([data_prov_milan, data_prov_varese,data_prov_como,data_prov_lecco,data_prov_sondrio,data_prov_bergamo,data_prov_brescia,data_prov_pavia,data_prov_cremona,data_prov_mantova,data_prov_lodi,data_prov_Monza]).reset_index(drop=True)

# Display the merged DataFrame
merged_df.rename(columns={'cod_prov':'COD_PROV'}, inplace=True)


# Creating dataframe of shapefiles

In [7]:
shapefile_path_region = "./Limiti01012024_g/Reg01012024_g/Reg01012024_g_WGS84.shp"
shapefile_path_province = "./Limiti01012024_g/ProvCM01012024_g/ProvCM01012024_g_WGS84.shp"
gdf_region = gpd.read_file(shapefile_path_region)
gdf_province = gpd.read_file(shapefile_path_province)

# filter dataframe with desire columns 

In [8]:

desired_cod_RIP = [15,12,13,97,14,16,17,18,19,20,98,108]

# Filter the GeoDataFrame based on the desired cod_RIP values
filtered_gdf = gdf_province[gdf_province['COD_PROV'].isin(desired_cod_RIP)]

# Display the filtered DataFrame


In [9]:
desired_cod_REG = [3]

# Filter the GeoDataFrame based on the desired cod_RIP values
filtered_gdf2 = filtered_gdf[filtered_gdf['COD_RIP'].isin(desired_cod_REG)]

# Display the filtered DataFrame
print(filtered_gdf2)

Empty GeoDataFrame
Columns: [COD_RIP, COD_REG, COD_PROV, COD_CM, COD_UTS, DEN_PROV, DEN_CM, DEN_UTS, SIGLA, TIPO_UTS, Shape_Leng, Shape_Area, geometry]
Index: []


In [10]:
merged_df2 = merged_df.merge(filtered_gdf[['COD_PROV', 'geometry']], on='COD_PROV', how='left')

In [11]:
columns_to_select = ['geometry','cod_reg','cod_rip','COD_PROV','ar_kmq','nome','uid',
    'ar_fr_p3p4', 'ar_fr_p2', 'ar_fr_p1', 
    'pop_fr_p2', 'pop_fr_p1', 
    'ed_fr_p2', 'ed_fr_p1', 
    'ar_frp3p4p', 'popfrp3p4p', 
    'ed_fr_p3p4', 'edfrp3p4p'
]

# Select the columns
selected_df = merged_df2[columns_to_select]
selected_df2 = selected_df.copy()

# Creating DB

In [12]:
from config import db_name, user, password, host
# Database connection parameters
# with open('config.py', 'w') as f:
#     f.write(db_name = input(f'Enter the database name for the postgres:\n'))
#     f.write(user = input(f'Enter the user for the postgres:\n'))
#     f.write(password = input(f'Enter the Password for the postgres:\n'))
#     f.write(host = input(f'Enter the host for the postgres:\n'))

# db_name = "project"
# user = "postgres"
# password = "admin"
# host = "localhost"
print (db_name, user, password, host)
# db_name = "project"
# user = "postgres"
# password = "admin"
# host = "localhost"

# Connect to the default database to create the new database
conn = psycopg2.connect(dbname="postgres", user=user, password=password, host=host)
conn.autocommit = True # needed to create a db programmatically
cursor = conn.cursor()

# Create the database
try:
    cursor.execute(f"CREATE DATABASE {db_name};")
except: pass

conn.close()

se_proj se4g user localhost


# Creating table

In [13]:
# Create SQLAlchemy base
Base = declarative_base()
# Define the table schema
class Dataset(Base):
    __tablename__ = 'dataset'
    id = Column(Integer, primary_key=True, autoincrement=True)
    cod_reg = Column(Integer)
    cod_rip = Column(Integer)
    cod_prov = Column(Integer)
    ar_kmq = Column(Float)
    nome = Column(String)
    uid = Column(Integer)
    ar_fr_p3p4 = Column(Float)
    ar_fr_p2 = Column(Float)
    ar_fr_p1 = Column(Float)
    pop_fr_p2 = Column(Float)
    pop_fr_p1 = Column(Float)
    ed_fr_p2 = Column(Float)
    ed_fr_p1 = Column(Float)
    ar_frp3p4p = Column(Float)
    popfrp3p4p = Column(Float)
    ed_fr_p3p4 = Column(Float)
    edfrp3p4p = Column(Float)
    geometry = Column(Geometry('MULTIPOLYGON'))

# Create an engine and connect to the PostgreSQL database
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:5432/{db_name}')
Base.metadata.create_all(engine)

C:\Users\Mohd Ammar Mughees\AppData\Local\Temp\ipykernel_8864\1135081772.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


# complete the rows of table by information of API and shapefiles


In [14]:

#gdf = gpd.GeoDataFrame(selected_df2, geometry=gpd.GeoSeries.from_wkt(selected_df2['geometry']))

# Convert geometries to WKT format
#gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)
selected_df2['geometry'] = selected_df2['geometry'].apply(wkt.dumps)
# Insert the data into the PostgreSQL database
Session = sessionmaker(bind=engine)
session = Session()

for index, row in selected_df2.iterrows():
    dataset_entry = Dataset(
        cod_reg=row['cod_reg'],
        cod_rip=row['cod_rip'],
        cod_prov=row['COD_PROV'],
        ar_kmq=row['ar_kmq'],
        nome=row['nome'],
        uid=row['uid'],
        ar_fr_p3p4=row['ar_fr_p3p4'],
        ar_fr_p2=row['ar_fr_p2'],
        ar_fr_p1=row['ar_fr_p1'],
        pop_fr_p2=row['pop_fr_p2'],
        pop_fr_p1=row['pop_fr_p1'],
        ed_fr_p2=row['ed_fr_p2'],
        ed_fr_p1=row['ed_fr_p1'],
        ar_frp3p4p=row['ar_frp3p4p'],
        popfrp3p4p=row['popfrp3p4p'],
        ed_fr_p3p4=row['ed_fr_p3p4'],
        edfrp3p4p=row['edfrp3p4p'],
        geometry=row['geometry']
    )
    session.add(dataset_entry)

session.commit()
session.close()

In [15]:
# db is finished 